# Sporza wielermanager woman analysis

**Author**: Sebastiaan Vanuytven  
**Date**: 17/2/24  
**Purpose**: Trying to select the best 12 riders based on their top 25 performances last year as well as their current prices

## Load libraries

In [91]:
import pandas as pd
import numpy as np
from unidecode import unidecode
#import matplotlib.pyplot as plt

## load the data

In [67]:
prices = pd.read_csv('../data/Prices.csv',  index_col=0)
results = pd.read_csv('../data/Woman.csv', index_col=0)
pointsPerRace = pd.read_csv('../data/points.csv', index_col=0)

In [68]:
metadataRace = results.loc[:,"Classification":"Type"]
dfResults = results.iloc[:,0:25]
metadataRace.Classification = metadataRace.Classification.replace("niet-World Tour", "Niet-WT")

/var/folders/_y/13wzb19j3wj2x0r5ltghjpq1br4glx/T/ipykernel_13794/417758253.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadataRace.Classification = metadataRace.Classification.replace("niet-World Tour", "Niet-WT")


In [75]:
print(dfResults.loc[race].items())

In [88]:
# Initialize an empty dictionary to store rider positions
rider_positions = {}
rider_points_by_race = {}

# Iterate over each race
for race in dfResults.index:
    # Iterate over each position column
    for position, rider in dfResults.loc[race].items():
        # Check if rider exists (not NaN)
        if pd.notna(rider):
            # Append rider position for the race
            if rider not in rider_positions:
                rider_positions[rider] = []
            rider_positions[rider].append(position)
            # Append points earned for the race
            if rider not in rider_points_by_race:
                rider_points_by_race[rider] = []
            rider_points_by_race[rider].append(pointsPerRace.loc[int(position), metadataRace.loc[race]["Classification"]])

# Initialize lists to store data for DataFrame
rider_names = []
positions = []
positions_length = []
points_earned = []
points_by_race = []

# Extract data from rider_positions dictionary and calculate points earned
for rider, rider_position_list in rider_positions.items():
    rider_names.append(rider)
    positions.append(rider_position_list)
    positions_length.append(len(rider_position_list))
    total_points = sum(rider_points_by_race[rider])
    points_earned.append(total_points)
    points_by_race.append(rider_points_by_race[rider])

# Create a new DataFrame
df_rider_stats = pd.DataFrame({ 'top25Places': positions_length, 'totalPoints': points_earned, 'results': positions, 'pointsByRace': points_by_race}, index=rider_names)

# Display the DataFrame
print(df_rider_stats)

                    top25Places  totalPoints                     results  \
KOPECKY Lotte                 6          398          [1, 2, 1, 1, 7, 2]   
WIEBES Lorena                 8          377  [2, 1, 1, 2, 18, 3, 14, 1]   
BASTIANELLI Marta             6          231         [3, 2, 1, 3, 6, 24]   
NORSGAARD Emma                1           44                         [4]   
GEORGI Pfeiffer               7          198     [5, 9, 1, 11, 16, 8, 7]   
...                         ...          ...                         ...   
EWERS Veronica                1            6                        [14]   
KOPPENBURG Clara              1            3                        [23]   
GÅSKJENN Ingvild              1            3                        [24]   
FISHER-BLACK Niamh            1           17                        [10]   
KRAAK Amber                   1            4                        [22]   

                                      pointsByRace  
KOPECKY Lotte            [80, 64, 

In [92]:
formatted_names = []

for name in df_rider_stats.index:
    capitals = []
    non_capitals = []
    for word in name.split():
        if word.isupper():
            capitals.append(word.capitalize())
        else:
            non_capitals.append(word)
    formatted_name = ' '.join(non_capitals + capitals)
    formatted_name = unidecode(formatted_name)  # Remove accents
    formatted_names.append(formatted_name)
    
df_rider_stats.index = formatted_names

In [103]:
df = pd.merge(df_rider_stats, prices, left_index=True, right_index=True, how='left')
df.dropna(subset=['Price'], inplace=True)

In [108]:

df["pointsPerPrice"] = df["totalPoints"] / df["Price"]
df.sort_values(by='pointsPerPrice', ascending=False).head(50)

,top25Places,totalPoints,results,pointsByRace,Price,pointsPerPrice
Demi Vollering,8,533,"[17, 1, 1, 2, 2, 1, 1, 1]","[5, 80, 60, 80, 48, 80, 80, 100]",12.0,44.416667
Lorena Wiebes,8,377,"[2, 1, 1, 2, 18, 3, 14, 1]","[64, 60, 80, 48, 5, 52, 8, 60]",10.0,37.700000
Lotte Kopecky,6,398,"[1, 2, 1, 1, 7, 2]","[80, 64, 60, 100, 30, 64]",12.0,33.166667
Pfeiffer Georgi,7,198,"[5, 9, 1, 11, 16, 8, 7]","[36, 16, 80, 11, 6, 25, 24]",6.0,33.000000
Shirin Van Anrooij,8,212,"[1, 24, 9, 8, 5, 3, 15, 15]","[80, 3, 12, 25, 27, 52, 6, 7]",7.0,30.285714
Vittoria Guazzini,4,148,"[3, 7, 3, 4]","[39, 24, 52, 33]",5.0,29.600000
Elisa Balsamo,4,205,"[4, 4, 2, 2]","[44, 33, 64, 64]",7.0,29.285714
Maike Van Der Duin,4,138,"[3, 7, 3, 13]","[52, 24, 52, 10]",5.0,27.600000
Elisa Longo Borghini,6,176,"[10, 11, 3, 21, 17, 2]","[14, 8, 65, 4, 5, 80]",7.0,25.142857
Marlen Reusser,7,224,"[14, 15, 1, 7, 7, 6, 3]","[6, 4, 80, 18, 30, 21, 65]",10.0,22.400000


#### Selection:

**Certitudes**
- Demi Vollering
- Lorena Wiebes
- Lotte Kopecky
- Marthe Truyen
- Pfeiffer Georgi
- Elise Chabbey
- Marthe Goossens
- Vittoria Guazzini
- CONSONNI Chiara
- Shirin Van Anrooij
- Karlijn Swinkels
- Margaux Vigie

**NA Roubaix**

- Transfer of Wiebes => Liana Lippert/Silvia Persico(9M)
- CONSONNI Chiara => Gaia Realini (7)
- Margaux Vigie/Karlijn Swinkels => Evita Muzic(5)

Klim Klassiekers:
- VOLLERING Demi
- CHABBEY Elise
- VAN ANROOIJ Shirin
- KOPECKY Lotte
- GEORGI Pfeiffer
- Liana Lippert/Silvia Persico
- Gaia Realini	
- Evita Muzic